## Biblioteca importadas

In [1]:

import numpy as np 
import pandas as pd
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt




## Leitura das bases de dados de cada aeroportos


#### Frequencia de passageiros no aeroporto CWB

In [2]:
#nacional 
filename = r'C:Passageiros pagos\cwb_nac.xlsx'
cwb_nac = pd.read_excel(filename).drop(["Unnamed: 0"],axis=1)

#internacional
filename = r'C:Passageiros pagos\cwb_int.xlsx'
cwb_int = (pd.read_excel(filename)
 .drop(["Unnamed: 0"],axis=1)
 .rename(columns ={'Passageiros':'Passageiros internacionais'})
          )


#### Frequencia de passageiros no aeroporto IGU

In [3]:
#nacional
filename = r'C:Passageiros pagos\igu_nac.xlsx'
igu_nac = pd.read_excel(filename).drop(["Unnamed: 0"],axis=1)

#internacional
filename = r'C:Passageiros pagos\igu_int.xlsx'
igu_int = (pd.read_excel(filename)
 .drop(["Unnamed: 0"],axis=1)
 .rename(columns ={'Passageiros':'Passageiros internacionais'})
          )

#### Frequencia de passageiros no aeroporto NVT

In [4]:
#nacional
filename = r'C:Passageiros pagos\nvt_nac.xlsx'
nvt_nac = pd.read_excel(filename).drop(["Unnamed: 0"],axis=1)

#internacional
filename = r'C:Passageiros pagos\nvt_int.xlsx'
nvt_int = (pd.read_excel(filename)
 .drop(["Unnamed: 0"],axis=1)
 .rename(columns ={'Passageiros':'Passageiros internacionais'})
          )

#### Frequencia de passageiros no aeroporto PMW

In [5]:
filename = r'C:Passageiros pagos\pmw.xlsx'
pmw = pd.read_excel(filename).drop(["Unnamed: 0"],axis=1)

#### Frequencia de passageiros no aeroporto SLZ

In [6]:
filename = r'C:Passageiros pagos\slz.xlsx'
slz = pd.read_excel(filename).drop(["Unnamed: 0"],axis=1)

## Variaveis macro economicas

#### Os dados ja foram puxados em escala logaritmica

#### Valor do barril de petróleo

In [7]:
filename = r'C:Dados macros\statistic_id262858_opec-oil-price-annually-1960-2022.xlsx'
oil = pd.read_excel(filename)
oil.head()

,Ano,Oil
0,1960,1.63
1,1961,1.57
2,1962,1.52
3,1963,1.50
4,1964,1.45


#### Cotação do dolar

In [8]:
filename = r'C:Dados macros\Base de dados Dólar.xlsx'
dolar = pd.read_excel(filename)
dolar = dolar.drop(["valor"], axis =1)
dolar.head()

,Ano,Câmbio
0,2000,0.000000
1,2001,0.283438
2,2002,0.516534
3,2003,-0.184261
4,2004,-0.058498


#### Inflação do turismo no Brasil

In [9]:
filename = r'C:Dados macros\statistic_id270812_inflation-rate-in-brazil-2027.xlsx'
inflacao = pd.read_excel(filename)
inflacao = inflacao.drop(["valor"], axis =1)
inflacao.head()


,Ano,Inflação
0,1987,2.2833
1,1988,6.2912
2,1989,14.3072
3,1990,29.4773
4,1991,4.3277


#### PIB do Brasil e dos Estados dos aeroportos 

In [10]:
filename = r'C:Dados macros\PIB_brasil_freq.xlsx'
pib = pd.read_excel(filename)
pib = pib.drop(["valor"], axis =1)
pib.head()

,Ano,PIB real Brasil
0,2000,0.000000
1,2001,0.000472
2,2002,0.001028
3,2003,0.000387
4,2004,0.001911


# Curitiba

##### juntando os pontos macros e uma unico DataFrame

In [11]:
principal_cwb =(cwb_nac
 .join(cwb_int.set_index('Ano'),on='Ano')
 .join(oil.set_index('Ano'),on='Ano')
 .join(dolar.set_index('Ano'),on='Ano')
 .join(inflacao.set_index('Ano'),on='Ano')
 .join(pib.set_index('Ano'),on='Ano')
          )


#### Correlação entre os dados pré selecionados 

In [12]:
principal_cwb.loc[principal_cwb["Ano"] == 2019, "Passageiros_freq_defasagem"] = 0 
principal_cwb = principal_cwb.loc[principal_cwb["Ano"]<2020]

principal_cwb.drop(["Passageiros internacionais","Passageiros_frequencia", "Ano","numero_de_passageiros","Oil"],axis=1).corr()


,Passageiros_freq_defasagem,Câmbio,Inflação,PIB real Brasil
Passageiros_freq_defasagem,1.000000,-0.378978,-0.229385,0.427399
Câmbio,-0.378978,1.000000,0.077090,-0.058331
Inflação,-0.229385,0.077090,1.000000,-0.353719
PIB real Brasil,0.427399,-0.058331,-0.353719,1.000000


#### Dividindo a base de dados em treinamento e teste 

Nas duas células de códigos seguintes temos a parte de treinamento e teste. O treino foi definido do ano 2000 até 2017 e na parte foi colocado todos os anos de 2000 até 2019. A variavel Y(a ser prevista) foi colocado a frequencia dos passageiros e nas variaveis X (Cambio, PIB real do Brasil e Inflação) e foi retirada todas as outras.

In [13]:
# Para treinamento utilizamos de 2000 até 2017
treino = principal_cwb.loc[principal_cwb["Ano"] < 2017]

# Para teste utilizamos de 2000 até 2019
teste = principal_cwb.loc[principal_cwb["Ano"] >= 2017]

##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [14]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Inflação","Câmbio"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano", "numero_de_passageiros","Oil","Inflação","Câmbio"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [15]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.285
Model:                                OLS   Adj. R-squared:                  0.183
Method:                     Least Squares   F-statistic:                     2.788
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):             0.0957
Time:                            23:12:24   Log-Likelihood:                 62.864
No. Observations:                      17   AIC:                            -119.7
Df Residuals:                          14   BIC:                            -117.2
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

C:\Users\vitor\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2629: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\vitor\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  "anyway, n=%i" % int(n))


##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [16]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Inflação"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano", "numero_de_passageiros","Oil","Inflação"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [17]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.285
Model:                                OLS   Adj. R-squared:                  0.121
Method:                     Least Squares   F-statistic:                     1.731
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.210
Time:                            23:12:24   Log-Likelihood:                 62.871
No. Observations:                      17   AIC:                            -117.7
Df Residuals:                          13   BIC:                            -114.4
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [18]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Câmbio"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano", "numero_de_passageiros","Oil","Câmbio"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [19]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.419
Model:                                OLS   Adj. R-squared:                  0.285
Method:                     Least Squares   F-statistic:                     3.130
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):             0.0624
Time:                            23:12:24   Log-Likelihood:                 64.636
No. Observations:                      17   AIC:                            -121.3
Df Residuals:                          13   BIC:                            -117.9
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [20]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano", "numero_de_passageiros","Oil"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [21]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.419
Model:                                OLS   Adj. R-squared:                  0.226
Method:                     Least Squares   F-statistic:                     2.167
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.135
Time:                            23:12:24   Log-Likelihood:                 64.636
No. Observations:                      17   AIC:                            -119.3
Df Residuals:                          12   BIC:                            -115.1
Df Model:                               4                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

# Foz do Iguaçu

##### juntando os pontos macros e uma unico DataFrame

In [22]:
principal_igu =(igu_nac
 .join(igu_int.set_index('Ano'),on='Ano')
 .join(oil.set_index('Ano'),on='Ano')
 .join(dolar.set_index('Ano'),on='Ano')
 .join(inflacao.set_index('Ano'),on='Ano')
 .join(pib.set_index('Ano'),on='Ano')
          )

##### Correlação entre os dados pré selecionados 

In [23]:
principal_igu.loc[principal_igu["Ano"] == 2019, "Passageiros_freq_defasagem"] = 0 
principal_igu = principal_igu.loc[principal_igu["Ano"]<2020]

principal_igu.drop(["Passageiros internacionais","Passageiros_frequencia", "Ano","numero_de_passageiros","Oil"],axis=1).corr()


,Passageiros_freq_defasagem,Câmbio,Inflação,PIB real Brasil
Passageiros_freq_defasagem,1.000000,-0.362140,0.128012,0.282233
Câmbio,-0.362140,1.000000,0.077090,-0.058331
Inflação,0.128012,0.077090,1.000000,-0.353719
PIB real Brasil,0.282233,-0.058331,-0.353719,1.000000


#### Dividindo a base de dados em treinamento e teste

Nas duas células de códigos seguintes temos a parte de treinamento e teste. O treino foi definido do ano 2000 até 2017 e na parte foi colocado todos os anos de 2000 até 2019. A variavel Y(a ser prevista) foi colocado a frequencia dos passageiros e nas variaveis X (Cambio, PIB real do Brasil e Inflação) e foi retirada todas as outras.

In [24]:
treino = principal_igu.loc[principal_igu["Ano"] < 2017]
teste = principal_igu.loc[principal_igu["Ano"] >= 2017]

##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [25]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Inflação","Câmbio"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Inflação","Câmbio"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [26]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.164
Model:                                OLS   Adj. R-squared:                  0.045
Method:                     Least Squares   F-statistic:                     1.374
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.285
Time:                            23:12:24   Log-Likelihood:                 54.162
No. Observations:                      17   AIC:                            -102.3
Df Residuals:                          14   BIC:                            -99.82
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [27]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Inflação"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Inflação"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [28]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.170
Model:                                OLS   Adj. R-squared:                 -0.021
Method:                     Least Squares   F-statistic:                    0.8878
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.473
Time:                            23:12:25   Log-Likelihood:                 54.223
No. Observations:                      17   AIC:                            -100.4
Df Residuals:                          13   BIC:                            -97.11
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [29]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Câmbio"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Câmbio"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [30]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.194
Model:                                OLS   Adj. R-squared:                  0.008
Method:                     Least Squares   F-statistic:                     1.043
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.406
Time:                            23:12:25   Log-Likelihood:                 54.472
No. Observations:                      17   AIC:                            -100.9
Df Residuals:                          13   BIC:                            -97.61
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [31]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [32]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.205
Model:                                OLS   Adj. R-squared:                 -0.060
Method:                     Least Squares   F-statistic:                    0.7721
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.564
Time:                            23:12:25   Log-Likelihood:                 54.585
No. Observations:                      17   AIC:                            -99.17
Df Residuals:                          12   BIC:                            -95.00
Df Model:                               4                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

# Navegantes

##### juntando os pontos macros e uma unico DataFrame

In [33]:
principal_nvt =(nvt_nac
 .join(nvt_int.set_index('Ano'),on='Ano')
 .join(oil.set_index('Ano'),on='Ano')
 .join(dolar.set_index('Ano'),on='Ano')
 .join(inflacao.set_index('Ano'),on='Ano')
 .join(pib.set_index('Ano'),on='Ano')
          )

##### Correlação entre os dados pré selecionados 

In [34]:
 
principal_nvt = principal_nvt.loc[principal_nvt["Ano"]<2020]

principal_nvt.drop(["Passageiros internacionais","Passageiros_frequencia", "Ano","numero_de_passageiros","Oil"],axis=1).corr()


,Passageiros_freq_defasagem,Câmbio,Inflação,PIB real Brasil
Passageiros_freq_defasagem,1.000000,-0.136459,0.048770,0.207299
Câmbio,-0.136459,1.000000,0.077090,-0.058331
Inflação,0.048770,0.077090,1.000000,-0.353719
PIB real Brasil,0.207299,-0.058331,-0.353719,1.000000


#### Dividindo a base de dados em treinamento e teste

Nas duas células de códigos seguintes temos a parte de treinamento e teste. O treino foi definido do ano 2000 até 2017 e na parte foi colocado todos os anos de 2000 até 2019. A variavel Y(a ser prevista) foi colocado a frequencia dos passageiros e nas variaveis X (Cambio, PIB real do Brasil e Inflação) e foi retirada todas as outras.

In [35]:
treino = principal_nvt.loc[principal_nvt["Ano"] < 2017]
teste = principal_nvt.loc[principal_nvt["Ano"] >= 2017]

##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [36]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Câmbio", "Inflação"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Câmbio", "Inflação"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [37]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.158
Model:                                OLS   Adj. R-squared:                  0.038
Method:                     Least Squares   F-statistic:                     1.312
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.300
Time:                            23:12:25   Log-Likelihood:                 45.141
No. Observations:                      17   AIC:                            -84.28
Df Residuals:                          14   BIC:                            -81.78
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [38]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil", "Inflação"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil", "Inflação"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [39]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.244
Model:                                OLS   Adj. R-squared:                  0.069
Method:                     Least Squares   F-statistic:                     1.397
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.288
Time:                            23:12:25   Log-Likelihood:                 46.056
No. Observations:                      17   AIC:                            -84.11
Df Residuals:                          13   BIC:                            -80.78
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [40]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Câmbio"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil","Câmbio"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [41]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.390
Model:                                OLS   Adj. R-squared:                  0.249
Method:                     Least Squares   F-statistic:                     2.768
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):             0.0839
Time:                            23:12:25   Log-Likelihood:                 47.879
No. Observations:                      17   AIC:                            -87.76
Df Residuals:                          13   BIC:                            -84.43
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 
A variavel dependente utilizada foi o PIB

In [42]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Passageiros internacionais","Ano","numero_de_passageiros","Oil"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [43]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.461
Model:                                OLS   Adj. R-squared:                  0.282
Method:                     Least Squares   F-statistic:                     2.569
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):             0.0920
Time:                            23:12:25   Log-Likelihood:                 48.940
No. Observations:                      17   AIC:                            -87.88
Df Residuals:                          12   BIC:                            -83.71
Df Model:                               4                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

# Palmas

##### juntando os pontos macros e uma unico DataFrame

In [44]:
principal_pmw =(pmw
 .join(oil.set_index('Ano'),on='Ano')
 .join(dolar.set_index('Ano'),on='Ano')
 .join(inflacao.set_index('Ano'),on='Ano')
 .join(pib.set_index('Ano'),on='Ano')
          )

##### Correlação entre os dados pré selecionados 

In [45]:
principal_pmw.loc[principal_pmw["Ano"] == 2019, "Passageiros_freq_defasagem"] = 0 
principal_pmw= principal_pmw.loc[principal_pmw["Ano"]<2020]
principal_pmw.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil"],axis=1).corr()

,Passageiros_freq_defasagem,Câmbio,Inflação,PIB real Brasil
Passageiros_freq_defasagem,1.000000,-0.418795,0.546400,0.037045
Câmbio,-0.418795,1.000000,0.077090,-0.058331
Inflação,0.546400,0.077090,1.000000,-0.353719
PIB real Brasil,0.037045,-0.058331,-0.353719,1.000000


#### Dividindo a base de dados em treinamento e teste

Nas duas células de códigos seguintes temos a parte de treinamento e teste. O treino foi definido do ano 2000 até 2017 e na parte foi colocado todos os anos de 2000 até 2019. A variavel Y(a ser prevista) foi colocado a frequencia dos passageiros e nas variaveis X (Cambio, PIB real do Brasil e Inflação) e foi retirada todas as outras.

In [46]:
treino = principal_pmw.loc[principal_pmw["Ano"] < 2017]
teste = principal_pmw.loc[principal_pmw["Ano"] >= 2017]

##### Testando quais variaveis são as que mais correspondem com a regressão 



A variavel dependente utilizada foi o PIB

In [47]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil","Inflação","Câmbio"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil","Inflação","Câmbio"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [48]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.077
Model:                                OLS   Adj. R-squared:                 -0.055
Method:                     Least Squares   F-statistic:                    0.5838
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.571
Time:                            23:12:25   Log-Likelihood:                 20.768
No. Observations:                      17   AIC:                            -35.54
Df Residuals:                          14   BIC:                            -33.04
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 


As variaveis dependentes utilizada foram o PIB e Câmbio

In [49]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil", "Inflação"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil", "Inflação"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [50]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.194
Model:                                OLS   Adj. R-squared:                  0.008
Method:                     Least Squares   F-statistic:                     1.041
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.407
Time:                            23:12:25   Log-Likelihood:                 21.917
No. Observations:                      17   AIC:                            -35.83
Df Residuals:                          13   BIC:                            -32.50
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 


As variaveis dependentes utilizada foram o PIB e Inflação

In [51]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil","Câmbio"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil","Câmbio"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [52]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.135
Model:                                OLS   Adj. R-squared:                 -0.065
Method:                     Least Squares   F-statistic:                    0.6743
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.583
Time:                            23:12:25   Log-Likelihood:                 21.316
No. Observations:                      17   AIC:                            -34.63
Df Residuals:                          13   BIC:                            -31.30
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 


As variaveis dependentes utilizada foram o PIB, Inflação e o Câmbio

In [53]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [54]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.201
Model:                                OLS   Adj. R-squared:                 -0.066
Method:                     Least Squares   F-statistic:                    0.7534
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.575
Time:                            23:12:25   Log-Likelihood:                 21.991
No. Observations:                      17   AIC:                            -33.98
Df Residuals:                          12   BIC:                            -29.82
Df Model:                               4                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

# São Luiz

##### juntando os pontos macros e uma unico DataFrame

In [55]:
principal_slz =(slz
 .join(oil.set_index('Ano'),on='Ano')
 .join(dolar.set_index('Ano'),on='Ano')
 .join(inflacao.set_index('Ano'),on='Ano')
 .join(pib.set_index('Ano'),on='Ano')
          )

##### Correlação entre os dados pré selecionados 

In [56]:
principal_slz.loc[principal_slz["Ano"] == 2019, "Passageiros_freq_defasagem"] = 0 
principal_slz= principal_slz.loc[principal_slz["Ano"]<2020]
principal_slz.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil"],axis=1).corr()


,Passageiros_freq_defasagem,Câmbio,Inflação,PIB real Brasil
Passageiros_freq_defasagem,1.000000,-0.358100,0.157277,0.275966
Câmbio,-0.358100,1.000000,0.077090,-0.058331
Inflação,0.157277,0.077090,1.000000,-0.353719
PIB real Brasil,0.275966,-0.058331,-0.353719,1.000000


#### Dividindo a base de dados em treinamento e teste

Nas duas células de códigos seguintes temos a parte de treinamento e teste. O treino foi definido do ano 2000 até 2017 e na parte foi colocado todos os anos de 2000 até 2019. A variavel Y(a ser prevista) foi colocado a frequencia dos passageiros e nas variaveis X (Cambio, PIB real do Brasil e Inflação) e foi retirada todas as outras.

In [57]:
treino = principal_slz.loc[principal_slz["Ano"] < 2017]
teste = principal_slz.loc[principal_slz["Ano"] >= 2017]

##### Testando quais variaveis são as que mais correspondem com a regressão 

A variavel dependente utilizada foi o PIB

In [58]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil","Inflação","Câmbio"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil","Inflação","Câmbio"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [59]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.171
Model:                                OLS   Adj. R-squared:                  0.052
Method:                     Least Squares   F-statistic:                     1.439
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.270
Time:                            23:12:26   Log-Likelihood:                 53.344
No. Observations:                      17   AIC:                            -100.7
Df Residuals:                          14   BIC:                            -98.19
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 

As variaveis dependentes utilizada foram o PIB e a Inflação

In [60]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil","Câmbio"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil","Câmbio"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [61]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.201
Model:                                OLS   Adj. R-squared:                  0.016
Method:                     Least Squares   F-statistic:                     1.087
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.389
Time:                            23:12:26   Log-Likelihood:                 53.657
No. Observations:                      17   AIC:                            -99.31
Df Residuals:                          13   BIC:                            -95.98
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

##### Testando quais variaveis são as que mais correspondem com a regressão 

As variaveis dependentes utilizada foram o PIB e a Câmbio

In [62]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil","Inflação"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil","Inflação"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [63]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.174
Model:                                OLS   Adj. R-squared:                 -0.017
Method:                     Least Squares   F-statistic:                    0.9099
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.463
Time:                            23:12:26   Log-Likelihood:                 53.375
No. Observations:                      17   AIC:                            -98.75
Df Residuals:                          13   BIC:                            -95.42
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------


##### Testando quais variaveis são as que mais correspondem com a regressão 

As variaveis dependentes utilizada foram o PIB, Inflação e o Câmbio

In [64]:
y_train= treino["Passageiros_frequencia"].copy()
y_test= teste["Passageiros_frequencia"].copy()

X_train = treino.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil"],axis=1).copy()
X_test = teste.drop(["Passageiros_frequencia", "Ano","numero_de_passageiros","Oil"],axis=1).copy()

#### Analise com OLS

A regressão linear está analisando um periodo de tempo de 19 anos, se iniciando em 2000 até 2019. Foi utilizada a biblioteca OLS (statsmoldels)

In [65]:
import statsmodels.api as sm


x = sm.add_constant(X_train)

model = sm.OLS(y_train, x).fit()


print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Passageiros_frequencia   R-squared:                       0.207
Model:                                OLS   Adj. R-squared:                 -0.057
Method:                     Least Squares   F-statistic:                    0.7853
Date:                    Wed, 21 Dec 2022   Prob (F-statistic):              0.556
Time:                            23:12:26   Log-Likelihood:                 53.731
No. Observations:                      17   AIC:                            -97.46
Df Residuals:                          12   BIC:                            -93.30
Df Model:                               4                                         
Covariance Type:                nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------